# Lesson 4: ReRank

In [ ]:
import os
import weaviate
from dotenv import load_dotenv
from utils import dense_retrieval
from utils import print_result
from utils import keyword_search

load_dotenv()

auth_config = weaviate.auth.AuthApiKey(
    api_key=os.getenv("WEAVIATE_API_KEY")
)

client = weaviate.Client(
    url=os.getenv("WEAVIATE_API_URL"),
    auth_client_secret=auth_config,
    additional_headers={
        "X-Cohere-Api-Key": os.getenv("COHERE_API_KEY"),
    }
)

# Dense Retrieval

In [ ]:
query = "What is the capital of Canada?"

In [ ]:
dense_retrieval_results = dense_retrieval(query, client)

In [ ]:
print_result(dense_retrieval_results)

# Improving Keyword Search with ReRank

In [ ]:
query_1 = "What is the capital of Canada?"

In [ ]:
query_1 = "What is the capital of Canada?"
results = keyword_search(query_1,
                         client,
                         properties=["text", "title", "url", "views", "lang", "_additional {distance}"],
                         num_results=3
                        )

for i, result in enumerate(results):
    print(f"i:{i}")
    print(result.get('title'))
    print(result.get('text'))

In [ ]:
query_1 = "What is the capital of Canada?"
results = keyword_search(query_1,
                         client,
                         properties=["text", "title", "url", "views", "lang", "_additional {distance}"],
                         num_results=500
                        )

for i, result in enumerate(results):
    print(f"i:{i}")
    print(result.get('title'))


In [ ]:
import cohere


def rerank_responses(query, responses, num_responses=10):
    co = cohere.Client(os.environ['COHERE_API_KEY'])
    reranked_responses = co.rerank(
        query=query,
        documents=responses,
        top_n=num_responses,  # controls how many reranked docs to return
        model="rerank-english-v2.0"
    )
    return reranked_responses

In [ ]:
texts = [result.get('text') for result in results]
reranked_text = rerank_responses(query_1, texts)
print(reranked_text)

In [ ]:
for i, rerank_result in enumerate(reranked_text):
    print(f"i:{i}")
    print(f"{rerank_result}")
    print()

# Improving Dense Retrieval with ReRank

In [ ]:
query_2 = "Who is the tallest person in history?"
results = dense_retrieval(query_2,client)

In [ ]:
for i, result in enumerate(results):
    print(f"i:{i}")
    print(result.get('title'))
    print(result.get('text'))
    print()

In [ ]:
texts = [result.get('text') for result in results]
reranked_text = rerank_responses(query_2, texts)

In [ ]:
for i, rerank_result in enumerate(reranked_text):
    print(f"i:{i}")
    print(f"{rerank_result}")
    print()